In [1]:
import tensorflow as tf

In [2]:
train_dir=r"C:\Users\bisha\Downloads\chest_xray-20250726T110654Z-1-001\chest_xray\train"
test_dir=r"C:\Users\bisha\Downloads\chest_xray-20250726T110654Z-1-001\chest_xray\test"
valid_dir=r"C:\Users\bisha\Downloads\chest_xray-20250726T110654Z-1-001\chest_xray\val"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),  # resize images
    batch_size=32,
    color_mode="grayscale"  # since X-rays are grayscale
)

# Load validation data
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir,
    image_size=(224, 224),
    batch_size=32,
    color_mode="grayscale"
)

# Load test data
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    color_mode="grayscale"
)

Found 5226 files belonging to 2 classes.
Found 16 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


In [3]:
print("Class Names:", train_ds.class_names)

Class Names: ['NORMAL', 'PNEUMONIA']


In [4]:
from tensorflow.keras import layers

normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds   = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds  = test_ds.map(lambda x, y: (normalization_layer(x), y))


In [5]:
#Applying augmentation to prevent overfitting
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),       # flip left-right
    layers.RandomRotation(0.1),            # rotate ±10%
    layers.RandomZoom(0.1),                # zoom in/out
    layers.RandomContrast(0.1)             # change contrast
])

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [7]:
model = models.Sequential([
    data_augmentation,

    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,1)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')   # binary classification
])

C:\Users\bisha\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)